---
title: "10.6_ai"
format:
  html: default
toc: false
---


# 10.6 • Artificial Intelligence: Search, Uncertainty, Learning, and Decisions

“AI” is a toolbox:
- **Symbolic** (reasoning with rules),
- **Search** (A*, heuristics),
- **Probabilistic** (Bayes nets, HMMs),
- **Decision-making** (MDPs, RL),
- **Learning** (from data — see 10.5).

We’ll **implement**: BFS vs A*, admissible heuristics, value iteration for an MDP, and a tiny Bayesian network inference example.

In [ ]:
import numpy as np, heapq
import matplotlib.pyplot as plt
np.random.seed(11088)

## 1) Problem-solving as search
**State space** + **actions** + **goal test** + **costs** → path. We compare BFS and A* on a grid with obstacles.

**Heuristic admissibility**: `h(n) ≤ true_cost(n→goal)` ⇒ A* finds an optimal path.

In [ ]:
def bfs(grid, start, goal):
    H,W = grid.shape; from collections import deque
    Q=deque([start]); came={start:None}; vis={start}
    while Q:
        r,c=Q.popleft()
        if (r,c)==goal: break
        for dr,dc in [(1,0),(-1,0),(0,1),(0,-1)]:
            nr,nc=r+dr,c+dc
            if 0<=nr<H and 0<=nc<W and grid[nr,nc]==0 and (nr,nc) not in vis:
                came[(nr,nc)]=(r,c); vis.add((nr,nc)); Q.append((nr,nc))
    path=[]; cur=goal
    while cur is not None: path.append(cur); cur=came.get(cur)
    return path[::-1]

def astar(grid, start, goal):
    H,W = grid.shape
    def h(a,b): return abs(a[0]-b[0])+abs(a[1]-b[1])  # Manhattan: admissible on 4-neighbour grid
    openq=[(h(start,goal),0,start,None)]
    came={}; g={start:0}; vis=set()
    while openq:
        f, cost, cur, par = heapq.heappop(openq)
        if cur in vis: continue
        vis.add(cur); came[cur]=par
        if cur==goal: break
        for d in [(1,0),(-1,0),(0,1),(0,-1)]:
            nr,nc=cur[0]+d[0], cur[1]+d[1]
            if 0<=nr<H and 0<=nc<W and grid[nr,nc]==0:
                ng=cost+1
                if ng < g.get((nr,nc), 1e9):
                    g[(nr,nc)]=ng
                    heapq.heappush(openq,(ng+h((nr,nc),goal), ng, (nr,nc), cur))
    path=[]; cur=goal
    while cur is not None: path.append(cur); cur=came.get(cur)
    return path[::-1]

grid = np.zeros((25,25), int)
grid[8:18,12]=1; grid[13,12]=0  # wall with door
start, goal = (0,0), (24,24)
p_bfs = bfs(grid,start,goal)
p_astar = astar(grid,start,goal)

plt.figure(figsize=(5,5))
plt.imshow(grid, cmap='gray_r')
rb,cb=zip(*p_bfs); ra,ca=zip(*p_astar)
plt.plot(cb,rb,'b--',label='BFS')
plt.plot(ca,ra,'r-',label='A*')
plt.legend(); plt.title('BFS vs A*'); plt.show()

**Observation**: BFS ignores distance to goal; A* uses `h` to expand promising nodes first → fewer expansions and often shorter runtime.

**Exercise 1**: Replace Manhattan with a heuristic that overestimates by +3. Does A* remain optimal? Why not? (Because it violates admissibility.)

## 2) Decisions under uncertainty: Markov Decision Processes (MDPs)
An MDP is \( (S,A,P,R,\gamma) \). We compute an **optimal policy** with **value iteration**.

\[ V_{k+1}(s) = \max_a \sum_{s'} P(s'|s,a) [R(s,a,s') + \gamma V_k(s')] \]

Environment: 3×3 grid, slipping 10% sideways. Reward +1 at goal, −0.04 per step.

In [ ]:
S=[(r,c) for r in range(3) for c in range(3)]
A=[(1,0),(-1,0),(0,1),(0,-1)]
goal=(2,2); gamma=0.95
def step(s,a,slip=0.1):
    r,c=s; dr,dc=a
    main=(r+dr,c+dc); slip_alt=(r+dc,c+dr)  # rough side slip
    nxt=[]
    for i, (nr,nc) in enumerate([main, slip_alt]):
        if 0<=nr<3 and 0<=nc<3:
            p=1-slip if i==0 else slip
            nxt.append(((nr,nc), p))
        else:
            p=1-slip if i==0 else slip
            nxt.append(((r,c), p))
    return nxt
R=lambda s: 1.0 if s==goal else -0.04

V={s:0.0 for s in S}
for _ in range(200):
    V = {s: (0 if s==goal else max(sum(p*(R(s2)+gamma*V[s2]) for s2,p in step(s,a)) for a in A)) for s in S}

Pi={}
for s in S:
    if s==goal: Pi[s]=(0,0); continue
    Pi[s]=max(A, key=lambda a: sum(p*(R(s2)+gamma*V[s2]) for s2,p in step(s,a)))
np.array([[Pi[(r,c)] for c in range(3)] for r in range(3)])

**Exercise 2**: Increase slip to 0.3. Does the optimal policy become more cautious (e.g., hugging walls)? Discuss risk-sensitive modifications.

### Policy iteration vs. value iteration
- Policy iteration alternates **policy evaluation** and **policy improvement**; often fewer iterations.
- Value iteration performs a Bellman backup to optimality; simpler to code.

## 3) Tiny Bayesian network: probabilistic reasoning
Variables: `Smoking (S) → Cough (C)` and `Smoking (S) → CVD (D)`. Given observed `Cough=True`, infer `P(S)` and `P(D)`.
This shows **belief propagation** in the simplest form.

In [ ]:
P_S = {1:0.3, 0:0.7}
P_C_given_S = {1:{1:0.6, 0:0.4}, 0:{1:0.2, 0:0.8}}
P_D_given_S = {1:{1:0.15,0:0.85}, 0:{1:0.05,0:0.95}}

# Observe C=1, compute posterior P(S|C=1) ∝ P(C=1|S)P(S)
unnorm = {s: P_C_given_S[s][1]*P_S[s] for s in [0,1]}
Z = sum(unnorm.values())
post_S = {s:unnorm[s]/Z for s in [0,1]}

# Predictive for D given C=1: sum_s P(D|S=s)P(S=s|C=1)
P_D1 = sum(P_D_given_S[s][1]*post_S[s] for s in [0,1])
post_S, P_D1

**Why this matters**: many ‘AI’ tasks reduce to **structured uncertainty** (Bayes nets, HMMs). ML fits parameters; probabilistic AI lets you **reason** with them.

## 4) Where ML (10.5) and RL meet
- If transitions/rewards unknown, **RL** learns by interaction.
- If states partially observed, **POMDPs** (belief over states) — not covered here, but conceptual upgrade.

## Exercises
1. Change the A* heuristic to Euclidean distance. Still admissible? (Yes.) Compare node expansions (instrument your code).
2. Add a pit state with −1 reward; recompute policy. How does risk trade-off appear?
3. Extend Bayes net: add `Activity (A)` that reduces `D` independently. Compute `P(D|C=1, A=1)`.

## Takeaways
- **Search** provides goal-directed behaviour without learning.
- **Probabilistic inference** manages uncertainty coherently.
- **Decision-making** optimises action with explicit trade-offs.
- **Learning** plugs in when models aren’t given; evaluation remains king.